In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.


# import kagglehub
# uwrfkaggler_ravdess_emotional_speech_audio_path = kagglehub.dataset_download('uwrfkaggler/ravdess-emotional-speech-audio')

# print('Data source import complete.')


## Load data

In [ ]:
import os
import librosa
import torch
from sklearn.preprocessing import LabelEncoder

dataset_path = "/kaggle/input/ravdess-emotional-speech-audio"

# Словарь эмоций
emotions_map = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}
# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised

# Функция загрузки аудиофайлов и меток
def load_data(dataset_path):
    audio_files = []
    labels = []
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.wav'):
                emotion_code = file.split('-')[2]  # Эмоция в имени файла
                if emotion_code in emotions_map:
                    labels.append(emotions_map[emotion_code])
                    audio_files.append(os.path.join(root, file))
    return audio_files, labels

# Загружаем данные
audio_files, labels = load_data(dataset_path)

# Кодируем эмоции
# le = LabelEncoder()
# labels_encoded = le.fit_transform(labels)



**Извлечение признаков из аудио:**
librosa: конвертируем аудио в спектрограммы MFCC (Mel-Frequency Cepstral Coefficients)

Используем предобученную Wav2Vec 2.0

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model

# Загружаем предобученную модель Wav2Vec 2.0
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

# Функция для извлечения признаков
def extract_wav2vec_features(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)  # Загружаем аудио
    inputs = processor(y, sampling_rate=sr, return_tensors="pt", padding=True)
    with torch.no_grad():
        features = model(inputs.input_values).last_hidden_state
    return features.mean(dim=1)  # Усредняем по времени

# Извлекаем признаки из всех файлов
wav2vec_features = [extract_wav2vec_features(path) for path in audio_files]
features_tensor = torch.cat(wav2vec_features)


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

## Создание классификатора эмоций

In [ ]:
# Здесь должен быть классификатор

Модель обучена на английских данных, а нам нужна русская речь

Fine-tuning на русских данных Golos

In [ ]:
# открываем данные Golos
def read_file_list(manifest):
    manifest_path, _ = os.path.split(manifest)
    files, texts = [], []
    with open(manifest, "r") as input_file:
        for line in input_file:
            as_dict = json.loads(line.rstrip('\n'))
            files.append(os.path.join(manifest_path, as_dict["audio_filepath"]))
            texts.append(as_dict["text"])
    return files, texts

In [ ]:
# Заменяем аудиофайлы RAVDESS на Golos
audio_files, labels = read_file_list .................

# Извлекаем признаки с помощью Wav2Vec 2.0
wav2vec_features = [extract_wav2vec_features(path) for path in audio_files]
features_tensor = torch.cat(wav2vec_features)

# Перекодируем метки эмоций
labels_encoded = le.transform(labels)
labels_tensor = torch.tensor(labels_encoded)

# Повторно обучаем классификатор

